# **Loan Default Prediction Challenge**

**Problem description**

SuperLender is a local digital lending company, which prides itself in its effective use of credit risk models to deliver profitable and high-impact loan alternative. Its assessment approach is based on two main risk drivers of loan default prediction:. 1) willingness to pay and 2) ability to pay. Since not all customers pay back, the company invests in experienced data scientist to build robust models to effectively predict the odds of repayment.

These two fundamental drivers need to be determined at the point of each application to allow the credit grantor to make a calculated decision based on repayment odds, which in turn determines if an applicant should get a loan, and if so - what the size, price and tenure of the offer will be.

There are two types of risk models in general: New business risk, which would be used to assess the risk of application(s) associated with the first loan that he/she applies. The second is a repeat or behaviour risk model, in which case the customer has been a client and applies for a repeat loan. In the latter case - we will have additional performance on how he/she repaid their prior loans, which we can incorporate into our risk model.

It is your job to predict if a loan was good or bad, i.e. accurately predict binary outcome variable, where Good is 1 and Bad is 0.

A good loan means the borrower paid it back in time, and a bad loan means the borrower defaulted because they have not made their scheduled payments for a predetermined amount of time.

In [433]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Let's first import some libraries.

In [434]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import classification_report
import joblib
from datetime import datetime, date
import calendar
from matplotlib import pyplot as plt
import seaborn as sns

Let's have a look at the data provided in our datasets:
First we load and store them in appropriately named variables.
**demo** contains demographic data of people taking out loans as follows:

a) Demographic data (traindemographics.csv)

*   customerid (Primary key used to merge to other data)
*   birthdate (date of birth of the customer)
*   bank_account_type (type of primary bank account)
*   longitude_gps
*   latitude_gps
*   bank_name_clients (name of the bank)
*   bank_branch_clients (location of the branch - not compulsory - so missing in a lot of the cases)
*   employment_status_clients (type of employment that customer has)
*   level_of_education_clients (highest level of education)

In [435]:
demo = pd.read_csv('/content/drive/MyDrive/Classroom/loan_data/traindemographics.csv')
demo.head()

,customerid,birthdate,bank_account_type,longitude_gps,latitude_gps,bank_name_clients,bank_branch_clients,employment_status_clients,level_of_education_clients
0,8a858e135cb22031015cbafc76964ebd,1973-10-10 00:00:00.000000,Savings,3.319219,6.528604,GT Bank,NaN,NaN,NaN
1,8a858e275c7ea5ec015c82482d7c3996,1986-01-21 00:00:00.000000,Savings,3.325598,7.119403,Sterling Bank,NaN,Permanent,NaN
2,8a858e5b5bd99460015bdc95cd485634,1987-04-01 00:00:00.000000,Savings,5.746100,5.563174,Fidelity Bank,NaN,NaN,NaN
3,8a858efd5ca70688015cabd1f1e94b55,1991-07-19 00:00:00.000000,Savings,3.362850,6.642485,GT Bank,NaN,Permanent,NaN
4,8a858e785acd3412015acd48f4920d04,1982-11-22 00:00:00.000000,Savings,8.455332,11.971410,GT Bank,NaN,Permanent,NaN


**perf** contains data of the repeat loan that the customer has taken for which we need to predict the performance of. This is the current loan data, but we are predicting whether this loan would default given all previous loans and demographics of the customer. It has the following descriptions:

b) Performance data (trainperf.csv) :

*   customerid (Primary key used to merge to other data)
*   systemloanid (The id associated with the particular loan. The same customerId can have multiple systemloanid’s for each loan he/she has taken out)
*   loannumber (The number of the loan that you have to predict)
*   approveddate (Date that loan was approved)
*   creationdate (Date that loan application was created)
*   loanamount (Loan value taken)
*   totaldue (Total repayment required to settle the loan - this is the capital loan value disbursed +interest and fees)
*   termdays (Term of loan)
*   referredby (customerId of the customer that referred this person - is missing, then not referred)
*   good_bad_flag (good = settled loan on time; bad = did not settled loan on time) - this is the target variable that we need to predict


In [436]:
perf = pd.read_csv('/content/drive/MyDrive/Classroom/loan_data/trainperf.csv')
perf.head()

,customerid,systemloanid,loannumber,approveddate,creationdate,loanamount,totaldue,termdays,referredby,good_bad_flag
0,8a2a81a74ce8c05d014cfb32a0da1049,301994762,12,2017-07-25 08:22:56.000000,2017-07-25 07:22:47.000000,30000.0,34500.0,30,NaN,Good
1,8a85886e54beabf90154c0a29ae757c0,301965204,2,2017-07-05 17:04:41.000000,2017-07-05 16:04:18.000000,15000.0,17250.0,30,NaN,Good
2,8a8588f35438fe12015444567666018e,301966580,7,2017-07-06 14:52:57.000000,2017-07-06 13:52:51.000000,20000.0,22250.0,15,NaN,Good
3,8a85890754145ace015429211b513e16,301999343,3,2017-07-27 19:00:41.000000,2017-07-27 18:00:35.000000,10000.0,11500.0,15,NaN,Good
4,8a858970548359cc0154883481981866,301962360,9,2017-07-03 23:42:45.000000,2017-07-03 22:42:39.000000,40000.0,44000.0,30,NaN,Good


**prev** has all previous loans that the customer had prior to the loan above (perf) that we want to predict the performance of. Each loan has a different systemloanid, but the same customerid for each customer.

c) Previous loans data (trainprevloans.csv) :

*   customerid (Primary key used to merge to other data)
*   systemloanid (The id associated with the particular loan. The same customerId can have multiple systemloanid’s for each loan he/she has taken out)
*   loannumber (The number of the loan that you have to predict)
*   approveddate (Date that loan was approved)
*   creationdate (Date that loan application was created)
*   loanamount (Loan value taken)
*   totaldue (Total repayment required to settle the loan - this is the capital loan value disbursed +interest and fees)
*   termdays (Term of loan)
*   closeddate (Date that the loan was settled)
*   referredby (customerId of the customer that referred this person - is missing, then not referred)
*   firstduedate (Date of first payment due in cases where the term is longer than 30 days. So in the case where the term is 60+ days - then there are multiple monthly payments due - and this dates reflects the date of the first payment)
*   firstrepaiddate (Actual date that he/she paid the first payment as defined above)



I think its important to note that the columns in perf and prev are the same in name save for **closed date, firstduedate** and **firstrepaiddate** in prev, and the **good_bad_flag** in perf.

In [437]:
prev = pd.read_csv('/content/drive/MyDrive/Classroom/loan_data/trainprevloans.csv')
prev.head()

,customerid,systemloanid,loannumber,approveddate,creationdate,loanamount,totaldue,termdays,closeddate,referredby,firstduedate,firstrepaiddate
0,8a2a81a74ce8c05d014cfb32a0da1049,301682320,2,2016-08-15 18:22:40.000000,2016-08-15 17:22:32.000000,10000.0,13000.0,30,2016-09-01 16:06:48.000000,NaN,2016-09-14 00:00:00.000000,2016-09-01 15:51:43.000000
1,8a2a81a74ce8c05d014cfb32a0da1049,301883808,9,2017-04-28 18:39:07.000000,2017-04-28 17:38:53.000000,10000.0,13000.0,30,2017-05-28 14:44:49.000000,NaN,2017-05-30 00:00:00.000000,2017-05-26 00:00:00.000000
2,8a2a81a74ce8c05d014cfb32a0da1049,301831714,8,2017-03-05 10:56:25.000000,2017-03-05 09:56:19.000000,20000.0,23800.0,30,2017-04-26 22:18:56.000000,NaN,2017-04-04 00:00:00.000000,2017-04-26 22:03:47.000000
3,8a8588f35438fe12015444567666018e,301861541,5,2017-04-09 18:25:55.000000,2017-04-09 17:25:42.000000,10000.0,11500.0,15,2017-04-24 01:35:52.000000,NaN,2017-04-24 00:00:00.000000,2017-04-24 00:48:43.000000
4,8a85890754145ace015429211b513e16,301941754,2,2017-06-17 09:29:57.000000,2017-06-17 08:29:50.000000,10000.0,11500.0,15,2017-07-14 21:18:43.000000,NaN,2017-07-03 00:00:00.000000,2017-07-14 21:08:35.000000


# **Preliminary understanding of the datasets' shapes.**

In [438]:
demo.dtypes

customerid                     object
birthdate                      object
bank_account_type              object
longitude_gps                 float64
latitude_gps                  float64
bank_name_clients              object
bank_branch_clients            object
employment_status_clients      object
level_of_education_clients     object
dtype: object

In [439]:
demo.shape

(4346, 9)

In [440]:
perf.dtypes

customerid        object
systemloanid       int64
loannumber         int64
approveddate      object
creationdate      object
loanamount       float64
totaldue         float64
termdays           int64
referredby        object
good_bad_flag     object
dtype: object

In [441]:
perf.shape

(4368, 10)

In [442]:
prev.dtypes

customerid          object
systemloanid         int64
loannumber           int64
approveddate        object
creationdate        object
loanamount         float64
totaldue           float64
termdays             int64
closeddate          object
referredby          object
firstduedate        object
firstrepaiddate     object
dtype: object

In [443]:
prev.shape

(18183, 12)

Some data cleaning.

In [444]:
demo.isnull().sum()

customerid                       0
birthdate                        0
bank_account_type                0
longitude_gps                    0
latitude_gps                     0
bank_name_clients                0
bank_branch_clients           4295
employment_status_clients      648
level_of_education_clients    3759
dtype: int64

In [445]:
demo_dup = demo.duplicated().any()
demo_dup

True

In [446]:
dup_count = demo.duplicated().sum()
dup_count

12

In [447]:
demo = demo.drop_duplicates()

In [448]:
demo.shape

(4334, 9)

In [449]:
demo = demo.drop(columns=['bank_branch_clients'])

In [450]:
demo.shape

(4334, 8)

In [451]:
demo.head()

,customerid,birthdate,bank_account_type,longitude_gps,latitude_gps,bank_name_clients,employment_status_clients,level_of_education_clients
0,8a858e135cb22031015cbafc76964ebd,1973-10-10 00:00:00.000000,Savings,3.319219,6.528604,GT Bank,NaN,NaN
1,8a858e275c7ea5ec015c82482d7c3996,1986-01-21 00:00:00.000000,Savings,3.325598,7.119403,Sterling Bank,Permanent,NaN
2,8a858e5b5bd99460015bdc95cd485634,1987-04-01 00:00:00.000000,Savings,5.746100,5.563174,Fidelity Bank,NaN,NaN
3,8a858efd5ca70688015cabd1f1e94b55,1991-07-19 00:00:00.000000,Savings,3.362850,6.642485,GT Bank,Permanent,NaN
4,8a858e785acd3412015acd48f4920d04,1982-11-22 00:00:00.000000,Savings,8.455332,11.971410,GT Bank,Permanent,NaN


In [452]:
dup_count1 = perf.duplicated().sum()
dup_count1

0

In [453]:
perf.isnull().sum()

customerid          0
systemloanid        0
loannumber          0
approveddate        0
creationdate        0
loanamount          0
totaldue            0
termdays            0
referredby       3781
good_bad_flag       0
dtype: int64

In [454]:
perf = perf.drop(columns=['referredby'])

In [455]:
perf.shape

(4368, 9)

In [456]:
dup_count2 = prev.duplicated().sum()
dup_count2

0

In [457]:
prev.isnull().sum()

customerid             0
systemloanid           0
loannumber             0
approveddate           0
creationdate           0
loanamount             0
totaldue               0
termdays               0
closeddate             0
referredby         17157
firstduedate           0
firstrepaiddate        0
dtype: int64

In [458]:
prev = prev.drop(columns=['referredby'])

In [459]:
prev.shape

(18183, 11)

As **'customerid'** is the primary key, I tried to ensure there were similar values in all the datasets.

In [460]:
demo['customerid'].isin(perf['customerid']).value_counts()

customerid
True     3269
False    1065
Name: count, dtype: int64

In [461]:
prev['customerid'].isin(perf['customerid']).value_counts()

customerid
True    18183
Name: count, dtype: int64

In [462]:
demo['customerid'].isin(prev['customerid']).value_counts()

customerid
True     3264
False    1070
Name: count, dtype: int64

From this information we can conclude that there are shared values in the customerid columns of demo, prev and perf datasets.

In [463]:
demo.columns

Index(['customerid', 'birthdate', 'bank_account_type', 'longitude_gps',
       'latitude_gps', 'bank_name_clients', 'employment_status_clients',
       'level_of_education_clients'],
      dtype='object')

In [464]:
perf.columns

Index(['customerid', 'systemloanid', 'loannumber', 'approveddate',
       'creationdate', 'loanamount', 'totaldue', 'termdays', 'good_bad_flag'],
      dtype='object')

In [465]:
prev.columns

Index(['customerid', 'systemloanid', 'loannumber', 'approveddate',
       'creationdate', 'loanamount', 'totaldue', 'termdays', 'closeddate',
       'firstduedate', 'firstrepaiddate'],
      dtype='object')

In [466]:
prev['systemloanid'].isin(perf['systemloanid']).value_counts()

systemloanid
False    18183
Name: count, dtype: int64

In [467]:
prev['approveddate'].isin(perf['approveddate']).value_counts()

approveddate
False    18182
True         1
Name: count, dtype: int64

In [468]:
prev['loannumber'].isin(perf['loannumber']).value_counts()

loannumber
True     13836
False     4347
Name: count, dtype: int64

While perf and prev have intersecting data in some columns, it is not enough to justify merging all of them. For this reason, we are going to rename the columns in one dataset to separate the information.

In [469]:
perf.columns = ['customerid','perfsystemloanid','perf_loannumber','perf_approveddate','perf_creationdate',
                'perf_loanamount','perf_totaldue','perf_termdays','good_bad_flag']

In [470]:
perf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4368 entries, 0 to 4367
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   customerid         4368 non-null   object 
 1   perfsystemloanid   4368 non-null   int64  
 2   perf_loannumber    4368 non-null   int64  
 3   perf_approveddate  4368 non-null   object 
 4   perf_creationdate  4368 non-null   object 
 5   perf_loanamount    4368 non-null   float64
 6   perf_totaldue      4368 non-null   float64
 7   perf_termdays      4368 non-null   int64  
 8   good_bad_flag      4368 non-null   object 
dtypes: float64(2), int64(3), object(4)
memory usage: 307.2+ KB


In [471]:
loandata = pd.merge(prev, perf, on='customerid')
loandata.head()

,customerid,systemloanid,loannumber,approveddate,creationdate,loanamount,totaldue,termdays,closeddate,firstduedate,firstrepaiddate,perfsystemloanid,perf_loannumber,perf_approveddate,perf_creationdate,perf_loanamount,perf_totaldue,perf_termdays,good_bad_flag
0,8a2a81a74ce8c05d014cfb32a0da1049,301682320,2,2016-08-15 18:22:40.000000,2016-08-15 17:22:32.000000,10000.0,13000.0,30,2016-09-01 16:06:48.000000,2016-09-14 00:00:00.000000,2016-09-01 15:51:43.000000,301994762,12,2017-07-25 08:22:56.000000,2017-07-25 07:22:47.000000,30000.0,34500.0,30,Good
1,8a2a81a74ce8c05d014cfb32a0da1049,301883808,9,2017-04-28 18:39:07.000000,2017-04-28 17:38:53.000000,10000.0,13000.0,30,2017-05-28 14:44:49.000000,2017-05-30 00:00:00.000000,2017-05-26 00:00:00.000000,301994762,12,2017-07-25 08:22:56.000000,2017-07-25 07:22:47.000000,30000.0,34500.0,30,Good
2,8a2a81a74ce8c05d014cfb32a0da1049,301831714,8,2017-03-05 10:56:25.000000,2017-03-05 09:56:19.000000,20000.0,23800.0,30,2017-04-26 22:18:56.000000,2017-04-04 00:00:00.000000,2017-04-26 22:03:47.000000,301994762,12,2017-07-25 08:22:56.000000,2017-07-25 07:22:47.000000,30000.0,34500.0,30,Good
3,8a2a81a74ce8c05d014cfb32a0da1049,301923941,10,2017-06-01 13:34:30.000000,2017-06-01 12:34:21.000000,20000.0,24500.0,30,2017-06-25 15:24:06.000000,2017-07-03 00:00:00.000000,2017-06-25 15:13:56.000000,301994762,12,2017-07-25 08:22:56.000000,2017-07-25 07:22:47.000000,30000.0,34500.0,30,Good
4,8a2a81a74ce8c05d014cfb32a0da1049,301954468,11,2017-06-28 10:58:34.000000,2017-06-28 09:58:25.000000,20000.0,24500.0,30,2017-07-25 08:14:36.000000,2017-07-31 00:00:00.000000,2017-07-25 08:04:27.000000,301994762,12,2017-07-25 08:22:56.000000,2017-07-25 07:22:47.000000,30000.0,34500.0,30,Good


In [472]:
loan_data = pd.merge(demo, loandata, on='customerid')
loan_data.head()

,customerid,birthdate,bank_account_type,longitude_gps,latitude_gps,bank_name_clients,employment_status_clients,level_of_education_clients,systemloanid,loannumber,...,firstduedate,firstrepaiddate,perfsystemloanid,perf_loannumber,perf_approveddate,perf_creationdate,perf_loanamount,perf_totaldue,perf_termdays,good_bad_flag
0,8a858e135cb22031015cbafc76964ebd,1973-10-10 00:00:00.000000,Savings,3.319219,6.528604,GT Bank,NaN,NaN,301943951,1,...,2017-07-05 00:00:00.000000,2017-07-04 17:59:36.000000,301964962,2,2017-07-05 14:29:48.000000,2017-07-05 13:29:42.000000,10000.0,13000.0,30,Good
1,8a858e275c7ea5ec015c82482d7c3996,1986-01-21 00:00:00.000000,Savings,3.325598,7.119403,Sterling Bank,Permanent,NaN,301929966,1,...,2017-07-07 00:00:00.000000,2017-07-10 08:42:44.000000,301972172,2,2017-07-10 21:21:46.000000,2017-07-10 20:21:40.000000,10000.0,13000.0,30,Bad
2,8a858e5b5bd99460015bdc95cd485634,1987-04-01 00:00:00.000000,Savings,5.746100,5.563174,Fidelity Bank,NaN,NaN,301931228,3,...,2017-07-10 00:00:00.000000,2017-07-11 10:02:11.000000,301976271,4,2017-07-13 15:40:27.000000,2017-07-13 14:40:19.000000,10000.0,13000.0,30,Good
3,8a858e5b5bd99460015bdc95cd485634,1987-04-01 00:00:00.000000,Savings,5.746100,5.563174,Fidelity Bank,NaN,NaN,301895253,1,...,2017-05-23 00:00:00.000000,2017-05-27 12:52:45.000000,301976271,4,2017-07-13 15:40:27.000000,2017-07-13 14:40:19.000000,10000.0,13000.0,30,Good
4,8a858e5b5bd99460015bdc95cd485634,1987-04-01 00:00:00.000000,Savings,5.746100,5.563174,Fidelity Bank,NaN,NaN,301919784,2,...,2017-06-12 00:00:00.000000,2017-06-08 11:03:40.000000,301976271,4,2017-07-13 15:40:27.000000,2017-07-13 14:40:19.000000,10000.0,13000.0,30,Good


In [473]:
loan_data.shape

(13673, 26)

In [474]:
loan_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13673 entries, 0 to 13672
Data columns (total 26 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   customerid                  13673 non-null  object 
 1   birthdate                   13673 non-null  object 
 2   bank_account_type           13673 non-null  object 
 3   longitude_gps               13673 non-null  float64
 4   latitude_gps                13673 non-null  float64
 5   bank_name_clients           13673 non-null  object 
 6   employment_status_clients   12310 non-null  object 
 7   level_of_education_clients  3464 non-null   object 
 8   systemloanid                13673 non-null  int64  
 9   loannumber                  13673 non-null  int64  
 10  approveddate                13673 non-null  object 
 11  creationdate                13673 non-null  object 
 12  loanamount                  13673 non-null  float64
 13  totaldue                    136

**'level_of_education_clients'** has too many null values to justify working with it further to train the model.



In [475]:
loan_data = loan_data.drop(columns=['level_of_education_clients'])

In [476]:
loan_data.shape

(13673, 25)

In [477]:
loan_data.head(15)

,customerid,birthdate,bank_account_type,longitude_gps,latitude_gps,bank_name_clients,employment_status_clients,systemloanid,loannumber,approveddate,...,firstduedate,firstrepaiddate,perfsystemloanid,perf_loannumber,perf_approveddate,perf_creationdate,perf_loanamount,perf_totaldue,perf_termdays,good_bad_flag
0,8a858e135cb22031015cbafc76964ebd,1973-10-10 00:00:00.000000,Savings,3.319219,6.528604,GT Bank,NaN,301943951,1,2017-06-19 17:55:26.000000,...,2017-07-05 00:00:00.000000,2017-07-04 17:59:36.000000,301964962,2,2017-07-05 14:29:48.000000,2017-07-05 13:29:42.000000,10000.0,13000.0,30,Good
1,8a858e275c7ea5ec015c82482d7c3996,1986-01-21 00:00:00.000000,Savings,3.325598,7.119403,Sterling Bank,Permanent,301929966,1,2017-06-07 12:47:30.000000,...,2017-07-07 00:00:00.000000,2017-07-10 08:42:44.000000,301972172,2,2017-07-10 21:21:46.000000,2017-07-10 20:21:40.000000,10000.0,13000.0,30,Bad
2,8a858e5b5bd99460015bdc95cd485634,1987-04-01 00:00:00.000000,Savings,5.746100,5.563174,Fidelity Bank,NaN,301931228,3,2017-06-08 11:49:34.000000,...,2017-07-10 00:00:00.000000,2017-07-11 10:02:11.000000,301976271,4,2017-07-13 15:40:27.000000,2017-07-13 14:40:19.000000,10000.0,13000.0,30,Good
3,8a858e5b5bd99460015bdc95cd485634,1987-04-01 00:00:00.000000,Savings,5.746100,5.563174,Fidelity Bank,NaN,301895253,1,2017-05-08 11:07:01.000000,...,2017-05-23 00:00:00.000000,2017-05-27 12:52:45.000000,301976271,4,2017-07-13 15:40:27.000000,2017-07-13 14:40:19.000000,10000.0,13000.0,30,Good
4,8a858e5b5bd99460015bdc95cd485634,1987-04-01 00:00:00.000000,Savings,5.746100,5.563174,Fidelity Bank,NaN,301919784,2,2017-05-27 17:10:41.000000,...,2017-06-12 00:00:00.000000,2017-06-08 11:03:40.000000,301976271,4,2017-07-13 15:40:27.000000,2017-07-13 14:40:19.000000,10000.0,13000.0,30,Good
5,8a858efd5ca70688015cabd1f1e94b55,1991-07-19 00:00:00.000000,Savings,3.362850,6.642485,GT Bank,Permanent,301939781,1,2017-06-15 14:05:26.000000,...,2017-06-30 00:00:00.000000,2017-07-18 19:14:05.000000,301997763,2,2017-07-26 21:03:17.000000,2017-07-26 20:03:09.000000,10000.0,11500.0,15,Good
6,8a858ea05a859123015a8892914d15b7,1990-07-21 00:00:00.000000,Savings,3.365935,6.564823,Access Bank,Permanent,301876300,3,2017-04-21 21:59:19.000000,...,2017-05-22 00:00:00.000000,2017-05-22 22:29:52.000000,301992494,6,2017-07-23 21:44:43.000000,2017-07-23 20:44:36.000000,20000.0,24500.0,30,Good
7,8a858ea05a859123015a8892914d15b7,1990-07-21 00:00:00.000000,Savings,3.365935,6.564823,Access Bank,Permanent,301953421,5,2017-06-27 13:07:32.000000,...,2017-07-27 00:00:00.000000,2017-07-21 21:47:08.000000,301992494,6,2017-07-23 21:44:43.000000,2017-07-23 20:44:36.000000,20000.0,24500.0,30,Good
8,8a858ea05a859123015a8892914d15b7,1990-07-21 00:00:00.000000,Savings,3.365935,6.564823,Access Bank,Permanent,301916071,4,2017-05-23 08:56:10.000000,...,2017-06-22 00:00:00.000000,2017-06-22 20:11:56.000000,301992494,6,2017-07-23 21:44:43.000000,2017-07-23 20:44:36.000000,20000.0,24500.0,30,Good
9,8a858ea05a859123015a8892914d15b7,1990-07-21 00:00:00.000000,Savings,3.365935,6.564823,Access Bank,Permanent,301849134,2,2017-03-29 11:10:12.000000,...,2017-04-28 00:00:00.000000,2017-04-21 21:30:48.000000,301992494,6,2017-07-23 21:44:43.000000,2017-07-23 20:44:36.000000,20000.0,24500.0,30,Good


Let's take note of the numerical and non-numerical columns to decide what approach to take next:

In [478]:
loan_data.columns

Index(['customerid', 'birthdate', 'bank_account_type', 'longitude_gps',
       'latitude_gps', 'bank_name_clients', 'employment_status_clients',
       'systemloanid', 'loannumber', 'approveddate', 'creationdate',
       'loanamount', 'totaldue', 'termdays', 'closeddate', 'firstduedate',
       'firstrepaiddate', 'perfsystemloanid', 'perf_loannumber',
       'perf_approveddate', 'perf_creationdate', 'perf_loanamount',
       'perf_totaldue', 'perf_termdays', 'good_bad_flag'],
      dtype='object')

In [479]:
categorical = [var for var in loan_data.columns if loan_data[var].dtype == 'O']
print('There are {} categorical variables \n'.format(len(categorical)))
print('They are: ', categorical)

There are 13 categorical variables 

They are:  ['customerid', 'birthdate', 'bank_account_type', 'bank_name_clients', 'employment_status_clients', 'approveddate', 'creationdate', 'closeddate', 'firstduedate', 'firstrepaiddate', 'perf_approveddate', 'perf_creationdate', 'good_bad_flag']


In [480]:
numerical = [var for var in loan_data.columns if loan_data[var].dtype != 'O']
print('There are {} numerical variables \n'.format(len(numerical)))
print('They are: ', numerical)

There are 12 numerical variables 

They are:  ['longitude_gps', 'latitude_gps', 'systemloanid', 'loannumber', 'loanamount', 'totaldue', 'termdays', 'perfsystemloanid', 'perf_loannumber', 'perf_loanamount', 'perf_totaldue', 'perf_termdays']


In [ ]:
loan_data[numerical].head()

In [505]:
loan_data[categorical].head()

,customerid,birthdate,bank_account_type,bank_name_clients,employment_status_clients,approveddate,creationdate,closeddate,firstduedate,firstrepaiddate,perf_approveddate,perf_creationdate,good_bad_flag
0,8a858e135cb22031015cbafc76964ebd,1973-10-10,Savings,GT Bank,Employment_unknown,2017-06-19 17:55:26.000000,2017-06-19 16:54:19.000000,2017-07-04 18:09:47.000000,2017-07-05 00:00:00.000000,2017-07-04 17:59:36.000000,2017-07-05 14:29:48.000000,2017-07-05 13:29:42.000000,Good
1,8a858e275c7ea5ec015c82482d7c3996,1986-01-21,Savings,Sterling Bank,Permanent,2017-06-07 12:47:30.000000,2017-06-07 11:46:22.000000,2017-07-10 08:52:54.000000,2017-07-07 00:00:00.000000,2017-07-10 08:42:44.000000,2017-07-10 21:21:46.000000,2017-07-10 20:21:40.000000,Bad
2,8a858e5b5bd99460015bdc95cd485634,1987-04-01,Savings,Fidelity Bank,Employment_unknown,2017-06-08 11:49:34.000000,2017-06-08 10:49:27.000000,2017-07-11 10:12:20.000000,2017-07-10 00:00:00.000000,2017-07-11 10:02:11.000000,2017-07-13 15:40:27.000000,2017-07-13 14:40:19.000000,Good
3,8a858e5b5bd99460015bdc95cd485634,1987-04-01,Savings,Fidelity Bank,Employment_unknown,2017-05-08 11:07:01.000000,2017-05-08 10:06:40.000000,2017-05-27 13:02:53.000000,2017-05-23 00:00:00.000000,2017-05-27 12:52:45.000000,2017-07-13 15:40:27.000000,2017-07-13 14:40:19.000000,Good
4,8a858e5b5bd99460015bdc95cd485634,1987-04-01,Savings,Fidelity Bank,Employment_unknown,2017-05-27 17:10:41.000000,2017-05-27 16:10:34.000000,2017-06-08 11:13:50.000000,2017-06-12 00:00:00.000000,2017-06-08 11:03:40.000000,2017-07-13 15:40:27.000000,2017-07-13 14:40:19.000000,Good


# Categorical data

In [481]:
loan_data['birthdate'] = pd.to_datetime(loan_data['birthdate'])

In [482]:
today = date.today()

In [483]:
today

datetime.date(2024, 4, 30)

In [484]:
loan_data['age'] = round(((datetime.now() - loan_data['birthdate'])/np.timedelta64(1, 'Y')),0)

In [485]:
loan_data.head()

,customerid,birthdate,bank_account_type,longitude_gps,latitude_gps,bank_name_clients,employment_status_clients,systemloanid,loannumber,approveddate,...,firstrepaiddate,perfsystemloanid,perf_loannumber,perf_approveddate,perf_creationdate,perf_loanamount,perf_totaldue,perf_termdays,good_bad_flag,age
0,8a858e135cb22031015cbafc76964ebd,1973-10-10,Savings,3.319219,6.528604,GT Bank,NaN,301943951,1,2017-06-19 17:55:26.000000,...,2017-07-04 17:59:36.000000,301964962,2,2017-07-05 14:29:48.000000,2017-07-05 13:29:42.000000,10000.0,13000.0,30,Good,51.0
1,8a858e275c7ea5ec015c82482d7c3996,1986-01-21,Savings,3.325598,7.119403,Sterling Bank,Permanent,301929966,1,2017-06-07 12:47:30.000000,...,2017-07-10 08:42:44.000000,301972172,2,2017-07-10 21:21:46.000000,2017-07-10 20:21:40.000000,10000.0,13000.0,30,Bad,38.0
2,8a858e5b5bd99460015bdc95cd485634,1987-04-01,Savings,5.746100,5.563174,Fidelity Bank,NaN,301931228,3,2017-06-08 11:49:34.000000,...,2017-07-11 10:02:11.000000,301976271,4,2017-07-13 15:40:27.000000,2017-07-13 14:40:19.000000,10000.0,13000.0,30,Good,37.0
3,8a858e5b5bd99460015bdc95cd485634,1987-04-01,Savings,5.746100,5.563174,Fidelity Bank,NaN,301895253,1,2017-05-08 11:07:01.000000,...,2017-05-27 12:52:45.000000,301976271,4,2017-07-13 15:40:27.000000,2017-07-13 14:40:19.000000,10000.0,13000.0,30,Good,37.0
4,8a858e5b5bd99460015bdc95cd485634,1987-04-01,Savings,5.746100,5.563174,Fidelity Bank,NaN,301919784,2,2017-05-27 17:10:41.000000,...,2017-06-08 11:03:40.000000,301976271,4,2017-07-13 15:40:27.000000,2017-07-13 14:40:19.000000,10000.0,13000.0,30,Good,37.0


In [495]:
age_pred = loan_data[['age','good_bad_flag']]
age_pred.head()

,age,good_bad_flag
0,51.0,Good
1,38.0,Bad
2,37.0,Good
3,37.0,Good
4,37.0,Good


In [489]:
loan_data['bank_account_type'].value_counts()

bank_account_type
Savings    8291
Other      5228
Current     154
Name: count, dtype: int64

In [493]:
loan_data['bank_name_clients'].value_counts()

bank_name_clients
GT Bank               5028
First Bank            1932
Access Bank           1268
UBA                   1116
Diamond Bank           983
Zenith Bank            948
Stanbic IBTC           489
EcoBank                450
FCMB                   420
Skye Bank              271
Fidelity Bank          267
Sterling Bank          111
Wema Bank               88
Heritage Bank           87
Keystone Bank           83
Union Bank              74
Standard Chartered      44
Unity Bank              14
Name: count, dtype: int64

From the earlier data we know that 'employment_status_clients' has several N/A values so let's work on that.

In [501]:
loan_data['employment_status_clients'].unique()

array([nan, 'Permanent', 'Student', 'Self-Employed', 'Unemployed',
       'Retired', 'Contract'], dtype=object)

In [502]:
loan_data['employment_status_clients'].fillna("Employment_unknown", inplace=True)

In [506]:
loan_data[categorical].head()

,customerid,birthdate,bank_account_type,bank_name_clients,employment_status_clients,approveddate,creationdate,closeddate,firstduedate,firstrepaiddate,perf_approveddate,perf_creationdate,good_bad_flag
0,8a858e135cb22031015cbafc76964ebd,1973-10-10,Savings,GT Bank,Employment_unknown,2017-06-19 17:55:26.000000,2017-06-19 16:54:19.000000,2017-07-04 18:09:47.000000,2017-07-05 00:00:00.000000,2017-07-04 17:59:36.000000,2017-07-05 14:29:48.000000,2017-07-05 13:29:42.000000,Good
1,8a858e275c7ea5ec015c82482d7c3996,1986-01-21,Savings,Sterling Bank,Permanent,2017-06-07 12:47:30.000000,2017-06-07 11:46:22.000000,2017-07-10 08:52:54.000000,2017-07-07 00:00:00.000000,2017-07-10 08:42:44.000000,2017-07-10 21:21:46.000000,2017-07-10 20:21:40.000000,Bad
2,8a858e5b5bd99460015bdc95cd485634,1987-04-01,Savings,Fidelity Bank,Employment_unknown,2017-06-08 11:49:34.000000,2017-06-08 10:49:27.000000,2017-07-11 10:12:20.000000,2017-07-10 00:00:00.000000,2017-07-11 10:02:11.000000,2017-07-13 15:40:27.000000,2017-07-13 14:40:19.000000,Good
3,8a858e5b5bd99460015bdc95cd485634,1987-04-01,Savings,Fidelity Bank,Employment_unknown,2017-05-08 11:07:01.000000,2017-05-08 10:06:40.000000,2017-05-27 13:02:53.000000,2017-05-23 00:00:00.000000,2017-05-27 12:52:45.000000,2017-07-13 15:40:27.000000,2017-07-13 14:40:19.000000,Good
4,8a858e5b5bd99460015bdc95cd485634,1987-04-01,Savings,Fidelity Bank,Employment_unknown,2017-05-27 17:10:41.000000,2017-05-27 16:10:34.000000,2017-06-08 11:13:50.000000,2017-06-12 00:00:00.000000,2017-06-08 11:03:40.000000,2017-07-13 15:40:27.000000,2017-07-13 14:40:19.000000,Good


Here, we are working with information I feel may be relevant to creating the model with regards to time.

1.   'creationdate' - 'closeddate', which shall give us the time from when the loan application was created, to when it was settled.
1.   'approveddate' - 'closeddate', which shall give us the time from when the loan was approved, to when it was settled.

Currently, I am considering checking between creation and approved date but I don't think there is any correlation in the data. We shall come back to this.

3.   'firstduedate' - 'firstrepaiddate' which shall give us the time from when the loan was first due, to when it was repaid.



In [507]:
loan_data['creationdate'] = pd.to_datetime(loan_data['creationdate'])
loan_data['closeddate'] = pd.to_datetime(loan_data['closeddate'])
loan_data['approveddate'] = pd.to_datetime(loan_data['approveddate'])
loan_data['firstrepaiddate'] = pd.to_datetime(loan_data['firstrepaiddate'])
loan_data['firstduedate'] = pd.to_datetime(loan_data['firstduedate'])

In [531]:
loan_data['creationlapse'] = ((loan_data.closeddate - loan_data.creationdate)/np.timedelta64(1, 'D'))
loan_data['creationlapse'] = loan_data['creationlapse'].round(1)

In [532]:
loan_data['approvedlapse'] = ((loan_data.closeddate - loan_data.approveddate)/np.timedelta64(1, 'D'))
loan_data['approvedlapse'] = loan_data['approvedlapse'].round(1)

In [533]:
loan_data['repaymentlapse'] = ((loan_data.firstduedate - loan_data.firstrepaiddate)/np.timedelta64(1, 'D'))
loan_data['repaymentlapse'] = loan_data['repaymentlapse'].round(1)

In [534]:
loan_data.head()

,customerid,birthdate,bank_account_type,longitude_gps,latitude_gps,bank_name_clients,employment_status_clients,systemloanid,loannumber,approveddate,...,perf_approveddate,perf_creationdate,perf_loanamount,perf_totaldue,perf_termdays,good_bad_flag,age,repaymentlapse,creationlapse,approvedlapse
0,8a858e135cb22031015cbafc76964ebd,1973-10-10,Savings,3.319219,6.528604,GT Bank,Employment_unknown,301943951,1,2017-06-19 17:55:26,...,2017-07-05 14:29:48.000000,2017-07-05 13:29:42.000000,10000.0,13000.0,30,Good,51.0,0.3,15.1,15.0
1,8a858e275c7ea5ec015c82482d7c3996,1986-01-21,Savings,3.325598,7.119403,Sterling Bank,Permanent,301929966,1,2017-06-07 12:47:30,...,2017-07-10 21:21:46.000000,2017-07-10 20:21:40.000000,10000.0,13000.0,30,Bad,38.0,-3.4,32.9,32.8
2,8a858e5b5bd99460015bdc95cd485634,1987-04-01,Savings,5.746100,5.563174,Fidelity Bank,Employment_unknown,301931228,3,2017-06-08 11:49:34,...,2017-07-13 15:40:27.000000,2017-07-13 14:40:19.000000,10000.0,13000.0,30,Good,37.0,-1.4,33.0,32.9
3,8a858e5b5bd99460015bdc95cd485634,1987-04-01,Savings,5.746100,5.563174,Fidelity Bank,Employment_unknown,301895253,1,2017-05-08 11:07:01,...,2017-07-13 15:40:27.000000,2017-07-13 14:40:19.000000,10000.0,13000.0,30,Good,37.0,-4.5,19.1,19.1
4,8a858e5b5bd99460015bdc95cd485634,1987-04-01,Savings,5.746100,5.563174,Fidelity Bank,Employment_unknown,301919784,2,2017-05-27 17:10:41,...,2017-07-13 15:40:27.000000,2017-07-13 14:40:19.000000,10000.0,13000.0,30,Good,37.0,3.5,11.8,11.8


creationlapse and approvedlapse basically have the same data. We shall go with approved lapse, which is the life/age of the loan.